# Contents
[1. How to use this notebook](#overview)

[2. What is functional annotation?](#en_pipeline)

[3. Importing your data](#en_import)

[4. Setting up your analysis environment](#en_env)

[5. Gene ID conversion](#en_ID)

[6. KEGG over-representation](#KEGG_or)

[7. Plotting KEGG results](#KEGG_plot)

[8. KEGG pathway maps](#KEGG_map)

[9. GO over-representation](#GO_or)

[10. Plotting GO results](#GO_plot)


# 1. How to use this notebook <a class="anchor" id="overview"></a>

This is a [Jupyter notebook](https://jupyter.org/) for functional annotation and analysis of a set of genes. 

Jupyter notebooks are interactive documents that contain 'live code', which allows the user to complete an analysis by running code 'cells', which can be modified, updated or added to by the user.

Individual Jupyter notebooks are based on a specific 'kernel', or analysis envirnment (mostly programming languages). This particular notebook is based on R. To see which version of R this notebook is based on, and as an example of running a code cell, click on the cell below and press the 'Run' button (top of the page).

In [ ]:
R.Version()

You should now see details about the version of R installed for this notebook. Every other code cell can be run similarly. There are two main types of code cells, plain R code (as seen above) and [markdown](https://www.rstudio.com/wp-content/uploads/2015/02/rmarkdown-cheatsheet.pdf). 

Markdown is a simple language for formatting text and the instructions (including this cell), headings, etc are written in markdown code cells. See: https://www.rstudio.com/wp-content/uploads/2015/02/rmarkdown-cheatsheet.pdf. You can add additional cells (markdown or R) by clicking on the plus sign, and then in the dropdown box, selecting 'Markdown' or 'code'. This way you can add your own analysis code cells or your own notes in markdown.

*******************************************

# 2. What is functional enrichment? <a class="anchor" id="en_pipeline"></a>

Many types of genetic analysis will output a set of genes that are associated with a specific experimental condition. The classic example of this is RNA-Seq, which outputs a set of genes that are differentially expressed between experimental conditions. But micro RNA, epigenetics (e.g. differential methylation), variant calling and various other analysis types can also generate a set of condition-based genes.

This Jupyter analysis workflow is designed so a researcher can input a set of genes from their analysis and use this to examine enrichment of these genes in [Kyoto Encyclopedia of Genes and Genomes (KEGG) pathways](https://www.genome.jp/kegg/) and [Gene Ontology (GO) terms](http://geneontology.org/).

### KEGG

[Kyoto Encyclopedia of Genes and Genomes (KEGG) database](https://www.genome.jp/kegg/)...

>.. is a database resource for understanding high-level functions and utilities of the biological system, such as the cell, the organism and the ecosystem, from genomic and molecular-level information. It is a computer representation of the biological system, consisting of molecular building blocks of genes and proteins (genomic information) and chemical substances (chemical information) that are integrated with the knowledge on molecular wiring diagrams of interaction, reaction and relation networks (systems information). It also contains disease and drug information (health information) as perturbations to the biological system.

![](https://www.genome.jp/kegg/docs/fig/kegg_overview.png)

### GO

[Gene Ontology (GO) database](http://geneontology.org/) ...

>...provides a computational representation of our current scientific knowledge about the functions of genes (or, more properly, the protein and non-coding RNA molecules produced by genes) from many different organisms, from humans to bacteria. It is widely used to support scientific research, and has been cited in tens of thousands of publications.

> Understanding gene function—how individual genes contribute to the biology of an organism at the molecular, cellular and organism levels—is one of the primary aims of biomedical research. Moreover, experimental knowledge obtained in one organism is often applicable to other organisms, particularly if the organisms share the relevant genes because they inherited them from their common ancestor.

>Associations of gene products to GO terms are statements that describe

>* Molecular Function: the molecular activities of individual gene products
>* Cellular Component: where the gene products are active
>* Biological Process: the pathways and larger processes to which that gene product’s activity contributes


**NOTE TO RESEARCHERS REGARDING ENRICHMENT RESULTS. PLEASE READ.** 

Enrichment estimations of KEGG pathways and GO terms are based on an over-representation test, which compares the complete set genes within all KEGG pathways or GO terms to the set of genes that you provide as input (e.g. a set of differentially expressed genes). If some of the genes in your set of input genes are overrepresented in spefific pathways, this pathway is considered 'enriched'. But since this is ratio-based, a small number of input genes can result in false positives. 

For example, if you use 4 genes as input and 2 of these genes are found in a specific KEGG pathway, this may result in this pathway being defined as significantly enriched, even though this is likely to be a false positive due to a small sample size (where sample size = number of genes). In addition, a single genes can be found in multiple pathways, thus having a small set of DE genes amplifies the chance of getting these false positives. 

**Always take into account the counts per pathway, gene ratio per pathway, total number of input genes. If there are a low number of gene counts in a pathway (e.g. < 5) and/or few genes used as input (e.g. < 20), then you may have false positive results.** Use your own judgement here. Talk to a bioinformatician if you want assistance in the interpratation of the results.

Also be aware that low numbers of DE genes or low gene counts per pathway will results in some strange looking barplots, dot plots, etc.

### R packages

A variety of R packages are used in this analysis workflow. You should cite these in any manuscripts that you write based on the results from this workflow.

Functional enrichment for KEGG pathways and GO terms was completed using the package [clusterProfiler](https://bioconductor.org/packages/release/bioc/html/clusterProfiler.html). You can read how to use clusterProfiler here: https://yulab-smu.top/biomedical-knowledge-mining-book/index.html

Annotated KEGG pathway maps are generated using the package [pathview](https://www.bioconductor.org/packages/release/bioc/html/pathview.html).

Annotation of primary gene IDs (e.g. gene symbol) to other gene identifiers (e.g. Entrez gene ID, Ensembl gene ID, description, etc) was completed using the [AnnotationHub package](https://bioconductor.org/packages/release/bioc/html/AnnotationHub.html).

*************************************************************

# 3. Importing your data <a class="anchor" id="en_import"></a>

Your data needs to be in a specific format and structure, so the R code in this workflow can identify and pull out the correct information.

1. Open Excel. Copy your **set of genes into column A** (e.g. a set of differentially expressed genes). This is technically all the information you need to identify enriched pathways. If you want to do some additional analysis (recommended) then you should add any **numeric data you have for your genes into column B** (e.g. log fold change scores) **and p values (or adjusted p values) to column C**. There should be no column headers. Save this file and call it "genes.xlsx" (note: R is case-sensitive, so name it exactly this, i.e. no capitals).

2. Copy this file to Jupyter. In the left hand panel you should see the 'File Browser' panel (or click on the 'File Browser' icon if it's not visible). You should see the 'Functional_annotation.ipynb' file here (i.e. this notebook).  Drag and drop the Excel file you created to the contents here.

**NOTE: There is a 'genes.xlsx' file here already, that contains a list of example genes. You can overwrite this with your own genes file, or continue to use the example data.**

*********************************************

# 4. Setting up your analysis environment <a class="anchor" id="en_env"></a>

Before you can begin your analysis you need to set up certain requirements, such as setting your working directory and installing/loading required R packges.

### Check your working directory

Your [working directory](https://r-coder.com/working-directory-r/) in R is a base directory where R looks for your data files (and outputs files to). Normally you'd need to set your working directory in R using the `setwd()` function. Here the working directory is where your Jupyter Notebook was downloaded to.

You can see what working directory you're currently in using the getwd() function:

In [ ]:
getwd()

Now you can see what is in this directory by running the dir function. Your Excel file should be here. i.e., if you're in the correct directory you should see your 'genes.xlsx' file and the Notebook (Functional_annotation.ipynb):

In [ ]:
dir()

### Install the required R packages

R does most of its analysis using [functions](https://www.tutorialspoint.com/r/r_functions.htm). Some of these are built into base R, but many come as external [packages](https://r-pkgs.org/intro.html), which need to be installed and loaded into R.

Packages are first installed in R using the `install.packages()` function and then loaded for use with the [library()](https://www.tutorialspoint.com/r/r_packages.htm) function:

In [ ]:
install.packages("tidyverse")

In [ ]:
library(tidyverse, quietly = TRUE)

In [ ]:
install.packages("readxl")

In [ ]:
library(readxl, quietly = TRUE)

In [ ]:
install.packages("scales")

In [ ]:
library(scales, quietly = TRUE)

Other packages need to be installed first.

The key R analysis package used is clusterProfiler (['A universal enrichment tool for interpreting omics data'](https://bioconductor.org/packages/release/bioc/html/clusterProfiler.html)). This package has multiple dependencies, so may take several minutes to install.

In [ ]:
install.packages("BiocManager", quietly = TRUE)

In [ ]:
BiocManager::install("clusterProfiler", quietly = TRUE)

In [ ]:
library(clusterProfiler, quietly = TRUE)

In [ ]:
BiocManager::install("pathview", quietly = TRUE)

In [ ]:
library(pathview, quietly = TRUE)

In [ ]:
BiocManager::install("AnnotationHub", quietly = TRUE)

In [ ]:
library(AnnotationHub, quietly = TRUE)

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(c("clusterProfiler", "pathview"), quietly = TRUE)
library(clusterProfiler, quietly = TRUE)
library(pathview, quietly = TRUE)

### Choose your species

To run the enrichment analysis, you need to download the genome annotation database for your species.

To do this, install the genome wide annotation package for your species (model organisms only). The list of species is here: https://bioconductor.org/packages/3.15/data/annotation/

Find your package name in the above link. E.g. for human data, search on that page for "Genome wide annotation for Human" and you'll see the package name is `"org.Hs.eg.db"`. Install this like so:

In [ ]:
BiocManager::install("org.Mm.eg.db", quietly = TRUE)
library(org.Mm.eg.db, quietly = TRUE)
myspeciesdb <- org.Mm.eg.db

If your species isn't human, replace "org.Hs.eg.db" in the above code cell with your species package name and run the code cell. E.g. mouse is `"org.Mm.eg.db"`, zebrafish is `"org.Dr.eg.db"`, rat is `"org.Rn.eg.db"`, etc.

You also need to define your species to match the correct KEGG database. The list of KEGG species is here: https://www.genome.jp/kegg/catalog/org_list.html. The first column is the KEGG species ID that you need. Run the following code cell to point to the human KEGG database.

As before, if your species isn't human, change this to your KEGG species ID and run the code cell. E.g. human is `"hsa"`, mouse is `"mmu"`, rat is `"rno'` etc.

In [ ]:
my_species <- "mmu"

### Other setup details

Choose a treatment name. In many experiments you will have multiple treatments, thus multiple sets of genes you want to examine. By giving your experiment a name here, any output files and figures will use that name, so that you're not overwriting previous treatment files and figures. If you only have one set of genes you can run this as the default (`treatment <- "single"`) but if you have different treatments, change this to a suitable treatment name (e.g. `treatment <- "highfat"`)

In [ ]:
treatment <- "single"

As a final optional setup for this notebook, you can set the default width and height for plots output here. You can modify the default width and height parameters here as you prefer.

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

**************************************

# 5. Gene ID conversion <a class="anchor" id="en_ID"></a>

KEGG pathways are based on [Entrez gene IDs](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1761442/), NCBIs numeric gene identifiers. Therefore, to run this workflow you need to convert your input gene IDs (usually gene symbols, but sometimes Ensembl or other IDs) into Entrez IDs.

First, read in the genes data you imported in section 3. 

In [ ]:
dat <- readxl::read_xlsx("genes.xlsx", col_names = F)

You can view the top 6 rows of your data like so:

In [ ]:
head(dat)

Now we can use the clusterProfiler `bitr` (**B**iological **I**d **TR**anslator) gene ID conversion function.

`OrgDb=..` points to the species orgDB package you installed in the previous section (e.g. `org.Hs.eg.db`), which we renamed to 'myspeciesdb'.
By default we're converting from gene symbol (`fromType="SYMBOL"`) to Entrez ID (`toType="ENTREZID"`). 

If you are starting with a different type of ID you can change the `fromType=` to your ID type. E.g. if you have a set of [Ensembl IDs](https://www.ebi.ac.uk/training/online/courses/ensembl-browsing-genomes/navigating-ensembl/investigating-a-gene/) you can change this to fromType="ENSEMBL".

In [ ]:
gene_list <- bitr(gene = dat$`...1`, fromType="SYMBOL", toType="ENTREZID", OrgDb=myspeciesdb, drop=TRUE)

You can only convert from and to the keytypes (i.e. gene/protein IDs) that are available for your species OrgDb package. You can view what keytypes are available for your species like so:

In [ ]:
keytypes(myspeciesdb)

**NOTE:** Not all gene IDs have an exact 1:1 match. E.g. a gene symbol may match to more than one Entrez ID. In addition, some of the IDs may not match at all. The output messages from bitr will tell you how many of your IDs failed to match and if there was a 1:1 match between ID types. If few or no genes matched, your set of gene IDs may be formatted incorrecty, you've chosen the wrong species, or you're not selecting the correct ID type. 

You can see how many ID's you have like so:

In [ ]:
nrow(gene_list)

Or you can see the first 6 genes like so:

In [ ]:
head(gene_list)

This set of Entrez gene IDs will be used as input for your enrichment analysis.

**********************

# 6. KEGG over-representation <a class="anchor" id="KEGG_or"></a>

Now we can run the clusterProfiler function `enrichKEGG` to match your set of Entrez IDs to the KEGG pathways for your species. 

You can adjust the `pvalueCutoff =` or `qvalueCutoff = 0.2` as needed. This will output only KEGG pathways that are *significantly* enriched, based on your p and q values. If you change these to 1 then this will output every KEGG pathway that contains any of your input Entrez gene IDs. You may prefer to run the fucntion this way first, then filter by q and p value later.

In [ ]:
kk <- enrichKEGG(gene = gene_list$ENTREZID, organism = my_species, pvalueCutoff = 0.05, qvalueCutoff = 0.2)

You can view a table of results like so:

In [ ]:
as.data.frame(kk)

The **ID** column is the KEGG pathway ID. The **GeneRatio** and **BgRatio** (background ratio) are the scores used to determine if a pathway is enriched (by overrepresentation). The **geneID** column lists the genes from your set of input genes that are found in that pathway. These are Entrez IDs. And the **Count** column is the number of genes from your set of input genes that are found in that pathway.

You can output this table as a comma separated file (can be opened in Excel or any text editor/viewer) like so:

In [ ]:
write.csv(as.data.frame(kk), paste0(treatment, "_KEGG_pathways.csv"))

The CSV file will appear in your working directory (the 'enrichment' directory you created earlier) and can be opened in Jupyter by double clicking on it in the left-hand 'File Browser' panel). You can also right click on the file to download it from the Jupyter server (or rename it or delete it, etc).

**********************

# 7. Plotting KEGG results <a class="anchor" id="KEGG_plot"></a>

clusterProfiler comes with a variety of built-in plotting functions. We'll use some of these and some custom plots to visualise the KEGG results.

### Bar plot

First we'll create a bar plot. This shows the pathway names on the Y axis, the X axis shows the number of your input genes ('count') found in each pathway, with bars coloured by adjusted p value.

By default the top 8 pathways, by adjusted p value, are shown (`showCategory = 8`), but you can change this to more or fewer pathways. You can also increase or decrease the font size.

In [ ]:
p <- barplot(kk, showCategory = 14, font.size = 14)
p

Now you can save the above plot as a 300dpi (i.e. publication quality) tiff or pdf file. **These files can be found in your working directory.**

**Tip:** you can adjust the width and height of the saved images by changing `width =` and `height =` in the code below. Pdf files can be opened within Jupyter, so a good way to test a suitable width/height would be to save the image by running the pdf code below with the default 20cm width/height, then open the pdf file by clicking on it in the file browser panel (left-hand panel), then change the width/height and repeat this process as needed.

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(treatment, "_KEGG_bar.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(treatment, "_KEGG_bar.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

### Dot plot

Now we'll create a dot plot. This plots the same data as the bar plot, but also includes gene ratio information (x axis). The pathway names are again on the Y axis, the size of the dots represents the number of your input genes ('count') found in each pathway, and dots coloured by adjusted p value.

As with the bar plot, you can adjust the number of pathways shown (`showCategory = ..`) and increase or decrease the font size. Note that the bar plot is ordered by adjusted p value but the dot plot is ordered by gene count, so the pathway order will differ in each plot.

In [ ]:
p <- dotplot(kk, showCategory = 8, font.size = 14)
p

Again, you can export these as a tiff or pdf file.

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(treatment, "_KEGG_dot.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(treatment, "_KEGG_dot.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

### Network plot

This plot shows a network of KEGG pathways - i.e. how pathways are connected based on which genes (from your input gene set) are shared between them.

There are several parameters you can modify here:

`showCategory = 8` change this to show more or fewer pathways, based on lowest ajusted p values.

`colorEdge = TRUE` will show different colour gene connection lines per pathway. You can change this to `FALSE` if you don't want different colours.

`node_label = "category"` will label just the pathways. If you change this to `node_label = "all"`, the genes will also be labelled (by Entrez gene ID).

In [ ]:
p <- cnetplot(kk, showCategory = 8, colorEdge = TRUE, node_label = "category")
p

You can also colour the genes using numeric data, such as log fold change. **This requires that you have provided numeric data in your original input dataset (see section 3). If you only provided a column of gene IDs then you can't generate the following plot and will have to use the above plot instead.**

To colour the genes you need to first match Entrez gene IDs to the numeric data you provided (e.g. log fold change) and then use this to create a named vector:

In [ ]:
gene_list_lfc <- merge(gene_list, dat, by.x = "SYMBOL", by.y = "...1", sort = F)
lfc2 <- gene_list_lfc$`...2`
names(lfc2) <- gene_list_lfc$ENTREZID

Then run the cnetplot() function again, adding the numeric data (`foldChange = lfc2`)

In [ ]:
p <- cnetplot(kk, showCategory = 8, colorEdge = TRUE, node_label = "category", foldChange = lfc2)
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(treatment, "_KEGG_network.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(treatment, "_KEGG_network.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**********************************

# 8. KEGG pathway maps <a class="anchor" id="KEGG_map"></a>

The package [pathview](https://www.bioconductor.org/packages/release/bioc/html/pathview.html) can pull down KEGG pathway maps from the KEGG database and annotate these with your input genes. It shades the genes by colour and intensity (e.g. using log fold change data, upregulated genes can be coloured green and downregulated coloured red).

**Note: this analysis needs numeric data, such as log fold change data, to work. If you only provided a set of gene IDs, with no numeric data (see section 3: Importing your data) then you will have to skip this section.**

pathview requires a named vector as input, so we first will pull out your numeric data (e.g. log fold change data) and name this using your gene IDs.

In [ ]:
lfc <- dat$`...2`
names(lfc) <- dat$`...1`

Then you need to select the pathway you want to annotate. Choose from one of the significantly enriched pathways identified in section 6. To view these:

In [ ]:
as.data.frame(kk)$ID

Enter one of these in the code cell below. You can change and re-run this and then the pathview code cell multiple times, to generate an annotated pathway map for each of your enriched KEGG pathways.

In [ ]:
keggpath <- "hsa04060"

Now run the pathview function to generate the pathway map. This will be output in your working directory. Double click to view it or right click to download.

In [ ]:
pview <- pathview(gene.data  = lfc,
                     pathway.id = keggpath,
                     species    = my_species,
                     gene.idtype = "SYMBOL",
                     low = list(gene = "red"),
                     mid = list(gene = "gray"),
                     high = list(gene = "green"),
                     out.suffix = treatment,
                     limit = list(gene=max(abs(lfc)), cpd=1))

**Note:** The above pathview command is optimised for differential expression data and thus colours pathway genes by log fold change, with upregulated genes being green and downregulated being red. If you have different numeric data you may want to modify the colour paremeters (`low = `, `mid = `, `high = `) to create a more suitable colour range. 

In addition, the command assumes you have gene symbol data as input. If you are using different gene IDs (Entrez, Ensembl, etc) then you need to change `gene.idtype = ".."` to your ID type. To see which ID types pathview can use as input, run the following code cell:

In [ ]:
data(gene.idtype.list)
gene.idtype.list

**********************************

# 9. GO over-representation <a class="anchor" id="GO_or"></a>

GO term overrepresetation is very similar to KEGG, in terms of statistical analysis (i.e. using a hypergeometric test) and plotting. The main difference is that GO terms are hierarchical and are separated by three top-level terms: **Biological Processes**, **Molecular Functions**, and **Cellular Components**. This means your set of input genes needs to be run 3 times, once for each top level term.

Fist select which of the top level terms you want to examine. You can run the analysis for each top-level term by changing this to another term then re-running this code cell and then the following GO code cells. The three terms are `"BP"`, `"MF"`, and `"CC"`.

In [ ]:
ont <- "BP"

Then run the enrichGO function. Where enrichKEGG matched to the KEGG species ID, enrichGO matches to your species OrgDb that you imported in section 4 (that we renamed to 'myspeciesdb').

In [ ]:
gg <- enrichGO(gene = as.character(gene_list$ENTREZID), OrgDb = myspeciesdb, ont = ont, pvalueCutoff = 0.05, qvalueCutoff = 0.2)

You can view a table of results like so:

In [ ]:
as.data.frame(gg)

You can output this table as a comma separated file (can be opened in Excel or any text editor/viewer) like so:

In [ ]:
write.csv(as.data.frame(gg), paste0(treatment, "_", ont, "_GO_terms.csv"))

**********************************

# 10. Plotting GO results <a class="anchor" id="GO_plot"></a>

As with the KEGG results, you can plot a barplot, dot plot or network plot for GO terms. Alter the number of terms shown (`showCategory =`) and font size as you like. Make sure you run this section for each top level GO term.

**Bar plot**

In [ ]:
p <- barplot(gg, showCategory = 14, font.size = 14)
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(treatment, "_", ont, "_GO_bar.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(treatment, "_", ont, "_GO_bar.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**Dot plot**

In [ ]:
p <- dotplot(gg, showCategory = 8, font.size = 14)
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(treatment, "_", ont, "_GO_dot.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(treatment, "_", ont, "_GO_dot.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**Network plot**

In [ ]:
p <- cnetplot(gg, showCategory = 8, colorEdge = TRUE, node_label = "category")
p

As with the KEGG network plots, you can also colour the genes per GO term by the numerical data (e.g. log fold change) you provided. If you didn't provide numerical data, skip the following plot:

In [ ]:
p <- cnetplot(gg, showCategory = 8, colorEdge = TRUE, node_label = "category", foldChange = lfc2)
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(treatment, "_", ont, "_GO_network.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(treatment, "_", ont, "_GO_network.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")

**Directed acyclic graph**

As previously mentioned, GO terms are hierarchical, with individual terms being linked to parent or child levels. The previous plots combine all levels under each of three top level terms. The following plot allows a visualisation of how the enriched GO terms fit into the GO hierarchy.

In [ ]:
p <- goplot(gg, showCategory = 30)
p

Export as a 300dpi tiff

In [ ]:
tiff_exp <- paste0(treatment, "_", ont, "_GO_DAG.tiff")
ggsave(file = tiff_exp, dpi = 300, compression = "lzw", device = "tiff", plot = p, width = 20, height = 20, units = "cm")

Export as a pdf

In [ ]:
pdf_exp <- paste0(treatment, "_", ont, "_GO_DAG.pdf")
ggsave(file = pdf_exp, device = "pdf", plot = p, width = 20, height = 20, units = "cm")